# Lab 2: Working with Anthropic API and Reasoning Models

### Overview

Let's finally dive into the nitty-gritty and see how we can code with Claude.

This notebook explores the Anthropic API, focusing on how to leverage both standard chat capabilities and advanced reasoning features. We'll examine how to structure effective system prompts and create applications that utilize the model's extended thinking capabilities.

### Objectives
- Set up and configure the Anthropic API
- Create tailored system prompts for specific use cases
- Compare standard responses with reasoning-enhanced outputs
- Analyze the "thinking" process in reasoning models

#### Step 1: Import libraries and load the environment variables

In [5]:
import os
from http.client import responses

import anthropic
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if ANTHROPIC_API_KEY is None:
    raise Exception("API key is missing")

#### Step 2: Call the API

In [2]:
client = anthropic.Client()

message = client.messages.create(
    model="claude-3-5-haiku-latest",
    max_tokens=100,
    system="You are a helpful assistant.",
    messages=[
        {"role": "user", "content": "What is the capital of Germany?"}
    ]
)

print(message)
print(message.content[0].text)

Message(id='msg_01Dg3wHNqSXP95dn8SLaYbFf', content=[TextBlock(citations=None, text='Berlin is the capital city of Germany. It is located in the northeastern part of the country and has been the capital since the reunification of Germany in 1990. Prior to that, during the period of division between East and West Germany, Bonn was the capital of West Germany, while East Berlin served as the capital of East Germany.', type='text')], model='claude-3-5-haiku-20241022', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=20, output_tokens=73, server_tool_use=None, service_tier='standard'))


#### Step 3: Customize the Prompt

Customer service representative

In [4]:
complaint = "You have over charged me for this product. I demand a refund."

In [5]:
company_policy = """
- We offer refunds only within 14 days of purchase.
- For purchases over $100, a supervisor must approve the refund.
- We prioritize customer satisfaction but aim to minimize refund abuse.

"""

In [8]:
system_message = "You are a helpful customer service representative."

prompt = f"""
Our client has made the following complaint:
{complaint}

Based on the company policy below, draft a professional and empathetic email response addressing the customer's concerns.

Company Policy:
{company_policy}
"""

client = anthropic.Client()

chat_message = client.messages.create(
    model="claude-3-5-haiku-latest",
    max_tokens=1000,
    temperature=0.3,
    system=system_message,
    messages=[
        {"role": "user", "content": prompt}
    ])

print(chat_message)
email = chat_message.content[0].text
print(email)


Message(id='msg_01UFTNbsiF2KD9irh35iqupr', content=[TextBlock(citations=None, text="Here's a draft email response:\n\nSubject: Regarding Your Refund Request\n\nDear Valued Customer,\n\nThank you for reaching out to us about your recent purchase. We take your concerns seriously and appreciate you bringing this matter to our attention.\n\nI understand you believe you have been overcharged for the product. To help resolve this, I would be happy to review the details of your purchase. Could you please provide me with:\n- Your order number\n- Date of purchase\n- Specific details about the pricing discrepancy you've noticed\n\nOur team will carefully investigate the charges and ensure they are accurate. If we find an error, we will process a refund promptly in accordance with our company policy.\n\nPlease note that our standard refund policy allows returns within 14 days of purchase. If your request falls within this timeframe, we will be glad to assist you further.\n\nWe value your satisfac

#### Step 4: Rate the response using a reasoning model

In [16]:
system_message = "You are a senior customer service agent with advanced reasoning capabilities. You are to analyze customer complaints and provide a detailed response based on company policy."

reasoning_prompt = f"""
A customer has made a complaint.

{complaint}

Evaluate this customer service email response.

{chat_message}

Company Policy:
{company_policy}

Based on this, decide how the support team should respond. Your analysis must include:
- Whether the response is appropriate given the complaint and company policy
- Professional tone and empathy

"""

reasoning_message = client.messages.create(
    model="claude-sonnet-4-5",
    thinking={
        'type': 'enabled',
        'budget_tokens': 2000
    },
    max_tokens=3000,
    temperature=1,
    system=system_message,
    messages=[
        {"role": "user", "content": reasoning_prompt}
    ]
)

reasoning_message = reasoning_message.content

print(reasoning_message)
print("\n")



[ThinkingBlock(signature='Eu4PCkYICxgCKkCKO8T3ffh4GMk6dVV6rAwoqqYYIbRVmEEjpfffNYy28vEFgZ9sas9mjmEFrnpCX8+x5tA/5/31JEI/YQgE2z7qEgzSAAEN3odgBomglQoaDPL/X4xnVK2RNcDuOSIwYvm+NpAN3xCTYZRFi87mJm24ClBJdrHB/DZVu6CE5JDktGJHl+CxTl6C1BRUC4WvKtUOMKeIHEO8pvQ7jenR5GCW4a6BmKxd3ChPzNB2UbpW7YGA1GpBavnCSWW9MopI6naC2icBeC0qTDj/DWkbNt8wI1W6B2PFTxamWoERjKAZ+rwqN5YM9+aG/xkGouhwL9yRrQuSPCno3RiTSd4OVMbpLnJ0q3qqT5BF8k+AfrVVa64hjDl0heVzOYjPSZKfmZ/ItnbSCvO20/aFvChvRKnQ+xjVAbfTX9Z5/koAiL8zD70JyWVsseiw0XrmRnj7JMhrB1QTBWyocidFEhph8aQ7STy4Gy7Z5ci/phXeGoaH6IacPx4jVEZTMqFvIBjrgRYpavbo9kIOvKqdkEaFUi0YCZBn6xyTMJ9L3m2IS7aYtLtxsc5wdYWXta2oCkNasNTeh8JyapqCBo5u0fmGjNscCPkkv99nSE+nOVzrfEMTveBSZM8kArZ7V2xEUvibHYlMsxBk2b8MlHydVrmG87A8nvbsdhH0qjtC1ZSBFfNkyxhb3axNdU5+GOLCVGUepSN6zvjw4mJlw4JjXVQnSU+ysMbXPvSqEsCbyYL2NLYye8wCduJQFpq9N8FwpFh9GxeysR4QxgpJZxGU0FaGqos5AR0yANQhPf2Tl/iF9mHdl57w3BsrszWAz2VFstQ+gIBWkY/mNzfue4vb7b0P2WWoLHDRrIH6C/JBuHjVnFT3p/ifiBPRMPEue6FcO5jY7J0MsdtP7gvlg9tQtXo6F+RiAS0OYXUy3cXEmTLepPTkj6rVNq6JuNtoPs54fEKjeG

In [22]:
# reasoning_message = reasoning_message[1].text

display(Markdown(reasoning_message[0].thinking))
print("\n------------------------------------------------------------\n")
display(Markdown(reasoning_message[1].text))


Let me analyze the customer service email response against the company policy and customer service best practices.

**Customer Complaint:**
- Brief and demanding: "You have over charged me for this product. I demand a refund."
- Customer is upset and claims overcharging
- Uses demanding language ("I demand")

**Company Policy:**
1. Refunds only within 14 days of purchase
2. Purchases over $100 require supervisor approval
3. Prioritize customer satisfaction but minimize refund abuse

**Evaluating the Response:**

**Strengths:**
1. Professional tone - acknowledges the concern without being defensive
2. Shows empathy - "We take your concerns seriously"
3. Requests necessary information (order number, date, pricing details)
4. Mentions the 14-day policy appropriately
5. Commits to investigation and prompt resolution
6. Uses polite language throughout
7. Offers personal follow-up

**Weaknesses:**
1. Doesn't distinguish between a "refund due to overcharging" vs. a standard return/refund
2. Doesn't mention the supervisor approval requirement for purchases over $100
3. Could better address the customer's emotional state (the demanding tone suggests frustration)
4. The phrase "if your request falls within this timeframe" could be misinterpreted - the 14-day policy may not apply to genuine pricing errors
5. Doesn't clarify what happens if there was an actual overcharge vs. if the customer simply wants to return the item

**Critical Issue:**
The response conflates two different scenarios:
- An overcharge (billing error) - which should be corrected regardless of return policy timeframes
- A standard refund/return request - which is subject to the 14-day policy

If the customer was genuinely overcharged, this is a billing error that should be corrected regardless of when it occurred. The 14-day return policy shouldn't apply to billing errors.


------------------------------------------------------------



# Analysis of Customer Service Email Response

## Overall Assessment: **Adequate but Needs Improvement**

The response demonstrates several strengths but contains critical gaps that could lead to confusion or escalation.

---

## Strengths

✅ **Professional Tone**
- Maintains composure despite the customer's demanding language
- Uses courteous phrases like "Valued Customer" and "Thank you for reaching out"
- No defensive or dismissive language

✅ **Appropriate Information Gathering**
- Requests essential details (order number, date, pricing discrepancy specifics)
- Shows willingness to investigate

✅ **Empathy Demonstrated**
- Acknowledges the concern: "We take your concerns seriously"
- Commits to personal review and resolution

---

## Critical Weaknesses

❌ **Conflates Two Different Scenarios**

The response treats an "overcharge" (billing error) the same as a standard refund request. These require different handling:

- **Overcharge/Billing Error**: Should be corrected regardless of timeframe
- **Standard Refund**: Subject to 14-day policy

**Problem:** The 14-day policy statement could imply that a genuine pricing error won't be corrected after 14 days, which is inappropriate.

❌ **Missing Policy Element**

- No mention of supervisor approval requirement for purchases over $100
- This could create delays or confusion later in the process

❌ **Insufficient De-escalation**

- The customer used demanding language ("I demand"), indicating frustration
- Response doesn't adequately validate their emotional state or apologize for the inconvenience

---

## Recommended Improvements

### Revised Key Section:

*"I understand you believe you were overcharged, and I sincerely apologize for any inconvenience this has caused. We take billing accuracy very seriously.*

*To resolve this promptly, I need to determine whether this is:*
1. *A pricing error on our end (which we will correct immediately)*
2. *A misunderstanding about the advertised price*
3. *A request to return the product*

*Could you please provide:*
- *Order number*
- *Amount you were charged vs. expected amount*
- *Where you saw the different price (if applicable)*

*If we made a billing error, we will issue a correction immediately, regardless of when the purchase occurred. For purchases over $100, our resolution may require supervisor review, but we will expedite this for you."*

---

## Compliance with Company Policy

| Policy Element | Addressed? | Notes |
|---------------|-----------|-------|
| 14-day refund window | ✅ Mentioned | Should clarify this applies to returns, not billing errors |
| Supervisor approval >$100 | ❌ Not mentioned | Critical omission |
| Minimize refund abuse | ✅ Implicit | Information gathering helps verify claim |
| Customer satisfaction priority | ✅ Demonstrated | Tone and commitment show this |

---

## Final Recommendation

**Rating: 6.5/10**

The response is professional and empathetic but needs refinement to:
1. Distinguish between billing errors and return requests
2. Include all policy requirements (supervisor approval)
3. Better address the customer's emotional state with a clear apology

With these adjustments, it would be an excellent response that balances customer satisfaction with policy compliance.

# **Your Challenge!!!**

A customer has submitted a complaint. Based on company policy and the situation, the model must select the correct resolution option from a list, and give a 1-line justification.

In [6]:
complaint = "I ordered a ceramic mug and received a broken one. Please fix this!"

policy = """
- Broken or damaged items must be reported within 7 days of delivery.
- If reported in time, the item is replaced or refunded.
- No action is taken for reports after 7 days unless escalated.
"""

metadata = """
    Reported Days After Delivery: 5
    Item Value: $12
    Customer Tier: Standard
    Past Complaints: 0
"""


Build a customer support agent that could help solve this issue

The agent should choose one of the following options:
- Refund Item
- Replace Item
- Offer Discount
- Apologize (no action)
- Report to supervisor

In [8]:
system_message = "You are a helpful customer service agent."

prompt = f"""A customer has made the following complaint:\n
{complaint}

Based on the company policy and the customer metadata below, choose the best resolution option from the list and provide a 1-line justification for your choice.
Company Policy:
{policy}

Customer Metadata:
{metadata}
"""

client = anthropic.Client()
message = client.messages.create(
    model="claude-sonnet-4-5",
    thinking={
        'type': 'enabled',
        'budget_tokens': 2000
    },
    max_tokens=3000,
    system=system_message,
    messages=[
        {"role": "user", "content": prompt}
    ])

In [9]:
display(Markdown(message.content[0].thinking))
print("\n------------------------------------------------------------\n")
display(Markdown(message.content[1].text))


Let me analyze this situation:

Customer Complaint: Received a broken ceramic mug

Company Policy:
- Broken/damaged items must be reported within 7 days of delivery
- If reported in time → replacement or refund
- Reports after 7 days → no action unless escalated

Customer Metadata:
- Reported: 5 days after delivery (within the 7-day window ✓)
- Item Value: $12 (low value)
- Customer Tier: Standard
- Past Complaints: 0 (good customer history)

Analysis:
- The customer reported within 5 days, which is within the 7-day policy window
- This is their first complaint (0 past complaints)
- The item value is relatively low ($12)
- According to policy, they qualify for replacement or refund

Best Resolution: Offer a replacement or refund

Justification: The customer reported within the required 7-day timeframe and has no history of complaints.


------------------------------------------------------------



**Best Resolution:** Offer replacement or refund for the broken mug

**Justification:** The customer reported within the required 7-day window (5 days) and qualifies for replacement or refund per company policy.

## **Reflections**

1. What reasoning steps did the model take to select its resolution? Were they logical and aligned with company policy? (Hint: Have a look at the reasoning output)
2. If you were designing the agent, what would you change about its decision-making process?
3. What did you learn about prompt design and reasoning models from this exercise?